In [1]:
import os
archivos_entrenamiento = os.listdir("Entrenamiento/")
archivo_test = os.listdir("Test/")
#set([os.path.splitext(archivo)[1] for archivo in archivos_entrenamiento]) # obtengo las extensiones de los archivos

In [2]:
class ArchivoTxt:
    def __init__(self, nombre, extension, texto):
        self.nombre = nombre
        self.extension = extension
        self.texto = texto

In [3]:
from tika import parser
from nltk import word_tokenize

def convertir_documento_a_txt(archivo):
    raw = parser.from_file(archivo)
    return raw['content']

#word_tokenize(convertir_pdf_txt("dataset-nlp-plagio-utn/Domótica_Final.pptx"))

In [4]:
def convertir_archivo_a_txt(archivo):
    archivo_nombre, archivo_extension = os.path.splitext(archivo)
    if(archivo_extension != ".pptx"):
        archivo_txt = convertir_documento_a_txt(archivo)
        return ArchivoTxt(archivo_nombre, archivo_extension, archivo_txt)
    
#convertir_archivo_a_txt("dataset-nlp-plagio-utn/MKT 2016 - Alan Szpigiel - TP4.pdf").nombre

In [6]:
from nltk import sent_tokenize # divide en oraciones

archivos_entrenamiento_txt = [convertir_archivo_a_txt("Entrenamiento/" + archivo) for archivo in archivos_entrenamiento]

#for archivo in archivos_entrenamiento_txt:
#    if(archivo != None):
#        archivo.texto = sent_tokenize(archivo.texto.strip())

In [7]:
archivo_test_txt = [convertir_archivo_a_txt("Test/" + archivo) for archivo in archivo_test][0]

In [8]:
# Nombre del archivo de texto procesado
archivo_test_txt.nombre # la extension hace falta?

'Test/TP 3 The experience economy'

In [10]:
# Nombre y apellido del alumno
import spacy
import itertools
from nltk import line_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords 

nombre_alumno=''

nlp=spacy.load('es_core_news_sm') # modelo para detectar Entidades (nombres)
texto_dividido_en_lineas = [re.sub('[^A-Za-z0-9áéíóúñ ]+', ' ', linea) for linea in line_tokenize(archivo_test_txt.texto.strip())]
lineas_pasadas_por_el_modelo = [[(a.text, a.label_) for a in nlp(linea).ents] for linea in texto_dividido_en_lineas]
lista_flatenizada = list(itertools.chain.from_iterable(lineas_pasadas_por_el_modelo)) #flatten

entidades_reconocidas_como_personas = [texto for (texto,categoria) in lista_flatenizada if categoria == 'PER']

sw = stopwords.words('spanish')
entidades_reconocidas_como_personas = [entidad for entidad in entidades_reconocidas_como_personas if not entidad.lower().strip() in sw]

#print([pos_tag(word_tokenize(entidad)) for entidad in entidades_reconocidas_como_personas])

for entidad in entidades_reconocidas_como_personas:
    if entidad in archivo_test_txt.nombre:
        nombre_alumno = entidad

if(nombre_alumno != ''):
    print('Nombre y apellido del alumno: ' + nombre_alumno)
else: # Fijarse cual de las entidades reconocidas como personas esta en un contexto como Nombre:, Integrantes:, etc
    print(entidades_reconocidas_como_personas)
    #print([nltk.Text(word_tokenize(archivo_test_txt.texto)).concordance(entidad) for entidad in entidades_reconocidas_como_personas])

['Alejandro Prince', 'Brian Daniel Leder', 'Joseph PINE II', 'James GILMORE', 'Trabajo Práctico', 'Cuáles', '9  Qué', 'adquisición']


In [9]:
import re, math
from collections import Counter

Palabra = re.compile(r'\w+')

def obtener_similitud_del_coseno(vector1, vector2):
     interseccion = set(vector1.keys()) & set(vector2.keys())
     numerador = sum([vector1[x] * vector2[x] for x in interseccion])

     sum1 = sum([vector1[x]**2 for x in vector1.keys()])
     sum2 = sum([vector2[x]**2 for x in vector2.keys()])
     denominador = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominador:
        return 0.0
     else:
        return float(numerador) / denominador

def string_a_vector(texto):
     palabras = Palabra.findall(texto)
     return Counter(palabras)

string1 = 'Le dije: Hola como te va Raúl?'
string2 = 'Le pregunté a Raúl como estaba'

def obtener_similitud(texto1, texto2):
    vector1 = string_a_vector(texto1)
    vector2 = string_a_vector(texto2)
    return obtener_similitud_del_coseno(vector1, vector2)

print('Similitud: ', obtener_similitud(string1, string2))

Similitud:  0.4629100498862757


In [11]:
def obtener_oracion_mas_parecida_del_dataset(oracion):
    mayor_porcentaje = 0.0
    oracion_mas_parecida = ''
    archivo_donde_se_encontro = ''
    for archivo in archivos_entrenamiento_txt:
        if(archivo != None and obtener_similitud(archivo_test_txt.texto,archivo.texto)<0.9):
            for oracion_a_comparar in sent_tokenize(archivo.texto.strip()):
                similitud = obtener_similitud(oracion.lower(), oracion_a_comparar.lower())
                if(similitud > mayor_porcentaje):
                    mayor_porcentaje = similitud
                    oracion_mas_parecida = oracion_a_comparar
                    archivo_donde_se_encontro = archivo.nombre
                
    return (re.sub('[^A-Za-z0-9áéíóúñ:/ ]+', ' ', oracion), re.sub('[^A-Za-z0-9áéíóúñ:/ ]+', ' ', oracion_mas_parecida), mayor_porcentaje, archivo_donde_se_encontro)

In [47]:
# Obtener Plagio de otros tps
# falta ver que tp pasó primero, que porcentaje poner como limite, que las consignas no sean plagio
porcentajes_de_aparicion = [obtener_oracion_mas_parecida_del_dataset(oracion) for oracion in sent_tokenize(archivo_test_txt.texto.strip())]
[(oracion,posible_plagio,porcentaje,archivo) for (oracion,posible_plagio,porcentaje,archivo) in porcentajes_de_aparicion if porcentaje>0.7]

[('Marketing en Internet y Nueva Economía curso: K5052    1 Cuat ',
  'Marketing en Internet y Nueva Economía curso: K5052    1 Cuat ',
  0.9999999999999998,
  'Entrenamiento/TP4 Adopcion (1)'),
 ('2017 Profesor: Alejandro Prince Fecha de entrega: 10/04/2017 NOMBRE Y APELLIDO LEGAJO N EMAIL CONTACTO Brian Daniel Leder 146 641 0 briandleder gmail com The experience economy   Joseph PINE II y James GILMORE Preguntas de Trabajo Práctico NOTA: no es requerida la lectura ni recordación desde la página 17 inclusive hasta la 20 del texto ',
  'Alumno: Andrés Basso                       TP 3: Economía de experiencia Legajo: 124 184 9 The experience economy   Joseph PINE II y James GILMORE Preguntas de Trabajo Práctico NOTA: no es requerida la lectura ni recordación desde la página 17 inclusive hasta la 20 del texto ',
  0.708010432393843,
  'Entrenamiento/TP 3 - Economía de Experiencia - Andrés Basso (1)'),
 ('Es obligatoria desde la 1ra hasta la página 16 ',
  'Es obligatoria desde la 1ra has

In [35]:
from bs4 import BeautifulSoup
import requests
requests.adapters.DEFAULT_RETRIES = 2

def obtener_html_como_texto(url):
    try:
        html = requests.get(url).text
    except requests.exceptions.ConnectionError:
        return ''
    soup = BeautifulSoup(html)
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [44]:
from googlesearch import search

def obtener_oracion_mas_parecida_de_internet(oracion):
    mayor_porcentaje = 0.0
    oracion_mas_parecida = ''
    url_donde_se_encontro = ''
    for url in search(re.sub('[^A-Za-z0-9áéíóúñ:/ ]+', ' ', oracion.lower()), tld="com.ar", num=2, stop=2):
        print('Buscando: ' + oracion + '\n En URL: ' + url + '\n')
        texto = obtener_html_como_texto(url)
        if texto != '':
            for oracion_a_comparar in sent_tokenize(texto.strip()):
                similitud = obtener_similitud(oracion.lower(), oracion_a_comparar.lower())
                if(similitud > mayor_porcentaje):
                    mayor_porcentaje = similitud
                    oracion_mas_parecida = oracion_a_comparar
                    url_donde_se_encontro = url
                
    return (re.sub('[^A-Za-z0-9áéíóúñ:/ ]+', ' ', oracion), re.sub('[^A-Za-z0-9áéíóúñ:/ ]+', ' ', oracion_mas_parecida), mayor_porcentaje, url_donde_se_encontro)

In [45]:
oracion_a_buscar = 'De cinco países para los que presenta datos, la consultora precisó que, a marzo, la Argentina fue el segundo más afectado, con una contracción el 9,8%'
obtener_oracion_mas_parecida_de_internet(oracion_a_buscar)

Buscando: De cinco países para los que presenta datos, la consultora precisó que, a marzo, la Argentina fue el segundo más afectado, con una contracción el 9,8%
 En URL: https://www.5dias.com.py/2020/05/la-economia-argentina-sera-la-mas-castigada-de-la-region/?utm_source=rss&utm_medium=rss&utm_campaign=la-economia-argentina-sera-la-mas-castigada-de-la-region

Buscando: De cinco países para los que presenta datos, la consultora precisó que, a marzo, la Argentina fue el segundo más afectado, con una contracción el 9,8%
 En URL: https://www.infobae.com/economia/2020/05/24/la-economia-argentina-sera-la-mas-castigada-de-la-region-por-el-efecto-de-las-cuarentenas/



('De cinco países para los que presenta datos  la consultora precisó que  a marzo  la Argentina fue el segundo más afectado  con una contracción el 9 8 ',
 'De cinco países para los que presenta datos  la consultora precisó que  a marzo  la Argentina fue el segundo más afectado  con una contracción el 9 8 ',
 1.0,
 'https://www.5dias.com.py/2020/05/la-economia-argentina-sera-la-mas-castigada-de-la-region/?utm_source=rss&utm_medium=rss&utm_campaign=la-economia-argentina-sera-la-mas-castigada-de-la-region')

In [46]:
# Obtener plagio de paginas de internet
porcentajes_de_aparicion = [obtener_oracion_mas_parecida_de_internet(oracion) for oracion in sent_tokenize(archivo_test_txt.texto.strip())]
[(oracion,posible_plagio,porcentaje,archivo) for (oracion,posible_plagio,porcentaje,archivo) in porcentajes_de_aparicion if porcentaje>0.7]

Buscando: Marketing en Internet y Nueva Economía
curso: K5052  -  1 Cuat.
 En URL: https://www.campusvirtual.frba.utn.edu.ar/especialidad/course/index.php?categoryid=106

Buscando: Marketing en Internet y Nueva Economía
curso: K5052  -  1 Cuat.
 En URL: https://rockcontent.com/es/blog/marketing-digital/

Buscando: 2017


Profesor: Alejandro Prince
Fecha de entrega: 10/04/2017

	NOMBRE Y APELLIDO
	LEGAJO N°
	EMAIL CONTACTO

	Brian Daniel Leder
	146.641-0
	briandleder@gmail.com














The experience economy - Joseph PINE II y James GILMORE
Preguntas de Trabajo Práctico
NOTA: no es requerida la lectura ni recordación desde la página 17 inclusive hasta la 20 del texto.
 En URL: https://www.genesys.com/fr-fr/blog/post/lemotion-vecteur-de-lexperience-client-part-1

Buscando: Es obligatoria desde la 1ra hasta la página 16.
 En URL: https://elpais.com/sociedad/2020-03-20/argentina-entra-en-cuarentena-obligatoria-hasta-el-31-de-marzo.html

Buscando: Es obligatoria desde la 1ra hasta la p

[]

In [47]:
[(oracion,posible_plagio,porcentaje,archivo) for (oracion,posible_plagio,porcentaje,archivo) in porcentajes_de_aparicion if porcentaje>0.4]

[('Marketing en Internet y Nueva Economía curso: K5052    1 Cuat ',
  'UTN BA   Aulas Virtuales: Marketing en Internet y Nueva Economía Salta al contenido principal Usted no se ha identificado ',
  0.43528575006600695,
  'https://www.campusvirtual.frba.utn.edu.ar/especialidad/course/index.php?categoryid=106'),
 ('Es obligatoria desde la 1ra hasta la página 16 ',
  'La idea es paralizar la actividad lo menos posible ',
  0.45454545454545453,
  'https://elpais.com/sociedad/2020-03-20/argentina-entra-en-cuarentena-obligatoria-hasta-el-31-de-marzo.html'),
 ('Diferencias con Producto y Servicio ',
  'Las 4 Diferencias entre Producto y Servicio en Marketing InicioCursosMarketing DigitalFacebook AdsTemas de MarketingEstrategiaDigitalPublicidadNoticiasRetailGuíasNewsletterAcerca del blogMis servicios Select Page  Cuál es la diferencia entre Producto y Servicio Marketing estratégico  Este artículo se trata sobre una diferencia fundamental dentro del marketing: la diferencia entre producto y ser